In [3]:
import pandas as pd

import sys
sys.path.append("../../scripts/")
from s3_support import *

Load the isolated transactions from auction forms that include offline transactions

In [5]:
df = get_dataframe_from_file("trans-records", "trans.auction_forms.csv")
df.head(3)

,id,org,form,recurring,recurring_creatingTransaction,creatingTransactionFor,pledge,state,zip,source,...,matching_gifts_count,sms_pledge_count,sms_pledge_amt,auction_purchase_count,auction_donation_count,hour,day,month,date,is_fraud
0,7727548,39915,948619,0,0,0,0,Indiana,46033,don_form,...,0,0,0.0,0,0,21.0,2.0,8.0,2019-08-21,False
1,7738294,169,948883,0,0,0,0,Florida,33811,don_form,...,0,0,0.0,0,0,17.0,4.0,8.0,2019-08-23,False
2,7740571,39915,948619,0,0,0,0,Indiana,46077,don_form,...,0,0,0.0,0,0,8.0,5.0,8.0,2019-08-24,False


_Notes from Joe: include extra tables; transpurchase, registration amount can go into transregistration, optional donation on registration goes into transdonation, check transrefund, also pull in matching and gift assist_

Transpurchase, transregistration, transdonation, & gift assist would all be represented in the transaction table but matching and deducted amounts for refunds have not yet been accounted for.

# spot checking form 953775

source reports: auction purchased items, auction donations

- donations: 25, \$2930, 7 declined
- purchases: 124 items, $63335
- total: \$66265
- checkins: 186
- total paid: 70

15 purchases w/ no attendee status, 1 checked in, 108 paid, 2 w/ amounts (totalling \$6400) with no transaction

In [23]:
this_df = df[df['form']==953775]
this_df = this_df[this_df['status'].isin(['A', 'CR', 'U'])]

print("transaction count: {}".format(len(this_df)))
print("transaction > $0: {}".format(len(this_df[this_df['amount']>0])))
print("total sum: ${}".format(this_df['amount'].sum()))

transaction count: 301
transaction > $0: 90
total sum: $59050.59


In [24]:
this_df['date'].min(), this_df['date'].max()

('2020-02-19', '2020-03-17')

In [25]:
this_df['date'] = pd.to_datetime(this_df['date'])
this_df_date = this_df[(this_df['date']>='2020-03-13')&(this_df['date']<='2020-03-14')]

print("transaction count: {}".format(len(this_df_date)))
print("transaction > $0: {}".format(len(this_df_date[this_df_date['amount']>0])))
print("total sum: ${}".format(this_df_date['amount'].sum()))

transaction count: 237
transaction > $0: 56
total sum: $38986.84


In [26]:
this_df.groupby('status')['amount'].sum()

status
A     54680.59
CR     4370.00
Name: amount, dtype: float64

In [31]:
q = "select * from transauction where form={} and status=1".format(953775)
transauction = redshift_query_read(q, schema='production')
q = "select * from transauctiondonation where form={} and status=1".format(953775)
transauctiondonation = redshift_query_read(q, schema='production')

In [32]:
print("auction count: {}".format(len(transauction)))
print("auction sum: ${:.2f}".format(transauction['total'].sum()))
print("unique trans auction count: {}".format(transauction.groupby('transaction')['total'].first().count()))
print("unique trans auction sum: ${:.2f}".format(transauction.groupby('transaction')['total'].first().sum()))

auction count: 184
auction sum: $99280.00
unique trans auction count: 61
unique trans auction sum: $37170.00


In [33]:
print("donations + purchases")
print("auction count: {} ({} purchases, {} donations)".format(len(transauction) + len(transauctiondonation), len(transauction), len(transauctiondonation)))
print("auction sum: ${:.2f}".format(transauction['total'].sum() + transauctiondonation['amount'].sum()))
print("unique trans auction count: {}".format(transauction.groupby('transaction')['total'].first().count() + transauctiondonation.groupby('transaction')['amount'].first().count()))
print("unique trans auction sum: ${:.2f}".format(transauction.groupby('transaction')['total'].first().sum() + transauctiondonation.groupby('transaction')['amount'].first().count()))

donations + purchases
auction count: 199 (184 purchases, 15 donations)
auction sum: $105955.00
unique trans auction count: 76
unique trans auction sum: $37185.00


In [34]:
# transaction referenced in auctions
auction_trans = transauction['transaction'].unique().tolist() + transauctiondonation['transaction'].unique().tolist()
this_df = df[df['id'].isin(auction_trans)]

print("transaction count: {}".format(len(this_df)))
print("transaction > $0: {}".format(len(this_df[this_df['amount']>0])))
print("total sum: ${}".format(this_df['amount'].sum()))

transaction count: 70
transaction > $0: 70
total sum: $56575.59


Comparing to dashboard reports (isolated to the date range 02/01/2020 - 04/01/2020):

- total: \$54733
- count: 299
- auction purchases: \$460
- auction bids: \$45060
- auction fund-a-need: \$6425
- auction total: \$51945

# spot checking form 948619

- purchases: 355, \$29,735
- donations: 20, \$9,312
- total: \$39,047

21 purchases w/out checked out status & no transaction, 35 checked out status (not paid)

In [10]:
this_df = df[df['form']==948619]
this_df = this_df[this_df['status'].isin(['A', 'CR'])]

print("transaction count: {}".format(len(this_df)))
print("transaction > $0: {}".format(len(this_df[this_df['amount']>0])))
print("total sum: ${}".format(this_df['amount'].sum()))

transaction count: 280
transaction > $0: 170
total sum: $50802.5


In [11]:
this_df.groupby('status')['amount'].sum()

status
A     50342.5
CR      460.0
Name: amount, dtype: float64

In [12]:
# 2019-10-12 - 2019-10-12
this_df['date'] = pd.to_datetime(this_df['date'])
this_df_date = this_df[(this_df['date']>='2019-10-12')&(this_df['date']<='2019-10-12')]

print("transaction count: {}".format(len(this_df_date)))
print("transaction > $0: {}".format(len(this_df_date[this_df_date['amount']>0])))
print("total sum: ${}".format(this_df_date['amount'].sum()))

transaction count: 120
transaction > $0: 62
total sum: $19405.0


In [13]:
this_df_date.groupby('status')['amount'].sum()

status
A    19405.0
Name: amount, dtype: float64

In [14]:
q = "select * from transauction where form={} and status=1".format(948619)
transauction = redshift_query_read(q, schema='production')
q = "select * from transauctiondonation where form={} and status=1".format(948619)
transauctiondonation = redshift_query_read(q, schema='production')

In [15]:
print("auction count: {}".format(len(transauction)))
print("auction sum: ${:.2f}".format(transauction['total'].sum()))
print("unique trans auction count: {}".format(transauction.groupby('transaction')['total'].first().count()))
print("unique trans auction sum: ${:.2f}".format(transauction.groupby('transaction')['total'].first().sum()))

auction count: 578
auction sum: $32160.00
unique trans auction count: 109
unique trans auction sum: $4635.00


In [16]:
print("donations + purchases")
print("auction count: {} ({} purchases, {} donations)".format(len(transauction) + len(transauctiondonation), len(transauction), len(transauctiondonation)))
print("auction sum: ${:.2f}".format(transauction['total'].sum() + transauctiondonation['amount'].sum()))
print("unique trans auction count: {}".format(transauction.groupby('transaction')['total'].first().count() + transauctiondonation.groupby('transaction')['amount'].first().count()))
print("unique trans auction sum: ${:.2f}".format(transauction.groupby('transaction')['total'].first().sum() + transauctiondonation.groupby('transaction')['amount'].first().count()))

donations + purchases
auction count: 623 (578 purchases, 45 donations)
auction sum: $44370.00
unique trans auction count: 154
unique trans auction sum: $4680.00


In [18]:
transauction['createddate'].min(), transauction['createddate'].max()

(Timestamp('2019-10-12 13:18:11'), Timestamp('2019-10-12 20:56:58'))

In [19]:
this_df['date'] = pd.to_datetime(this_df['date'])
this_df_date = this_df[(this_df['date']>='2019-10-12')&(this_df['date']<='2019-10-12')]

print("transaction count: {}".format(len(this_df_date)))
print("transaction > $0: {}".format(len(this_df_date[this_df_date['amount']>0])))
print("total sum: ${}".format(this_df_date['amount'].sum()))

transaction count: 120
transaction > $0: 62
total sum: $19405.0


In [21]:
# transaction referenced in auctions
auction_trans = transauction['transaction'].unique().tolist() + transauctiondonation['transaction'].unique().tolist()
this_df = df[df['id'].isin(auction_trans)]

print("transaction count: {}".format(len(this_df)))
print("transaction > $0: {}".format(len(this_df[this_df['amount']>0])))
print("total sum: ${}".format(this_df['amount'].sum()))

transaction count: 114
transaction > $0: 114
total sum: $28290.0


Comparing to dashboard reports (isolated to the date range 10/11/2019 - 10/13/2019):

- total: \$31330
- count: 187
- auction purchases: \$5825
- auction bids: \$9445
- auction fund-a-need: \$12110
- auction total: \$27380